# 初めに
Markdownモードは、ESC→M で変更できます。変更後、Alt+Enter で確定。  
セル左上の▷MLをクリックして\{\}にしてもOKっぽい。  
$$
\frac{1}{2}
$$
kerasを使った実装関係。  
VAEをkerasで実装  
https://qiita.com/iss-f/items/c43b23815fc6d15ae22e  
【Python】 VAE(Variational Auto Encoder)の写経  
http://ni4muraano.hatenablog.com/entry/2018/01/02/220000  
Variational Autoencoder徹底解説  
https://qiita.com/kenmatsu4/items/b029d697e9995d93aa24  

In [1]:
### エンコーダ(平均と分散を出力する簡単なモデル)
### 入力次元・出力次元はあらかじめ設定しておく
def build_encoder(self):
    x = Input(shape=(original_dim, ))

    hidden = Dense(intermediate_dim, activation='relu')(x)
    z_mean = Dense(latent_dim, activation='linear')(hidden)
    z_sigma = Dense(latent_dim, activation='linear')(hidden)
    return Model(x, [z_mean, z_sigma])

In [2]:
### デコーダ
def build_decoder(self):
    z_mean = Input(shape=(latent_dim ))
    z_sigma = Input(shape=(latent_dim, ))
    z = Lambda(self.sampling, output_shape=(latent_dim))([z_mean, z_sigma])  #Kerasのカスタムレイヤー
    h_decoded = Dense(intermediate_dim, activation='relu')(z)
    x_decoded_mean = Dense(original_dim, activation='sigmoid')(h_decoded)

    return Model([z_mean, z_sigma], x_decoded_mean)

def sampling(self, args):
    z_mean, z_sigma = args
    epsilon = K.random_normal(shape=(self.z_dim,), mean=0., stddev=epsilon_std)
    return z_mean + z_sigma * epsilon

In [3]:
    def build_vae(self, encoder, decoder):
        _, encoder_dense, encoder_mean, encoder_sigma = encoder.layers

        x = Input(shape=(self.input_dim, ))
        hidden = encoder_dense(x)
        z_mean = encoder_mean(hidden)
        z_sigma = encoder_sigma(hidden)

        self.z_m = z_mean # 損失関数で使う
        self.z_s = z_sigma # 損失関数で使う

        _, _, decoder_lambda, decoder_dense1, decoder_dense2 = decoder.layers
        z = decoder_lambda([z_mean, z_sigma])
        h_decoded = decoder_dense1(z)
        x_decoded_mean = decoder_dense2(h_decoded)
        return Model(x, x_decoded_mean)

In [4]:
### 損失関数
def binary_crossentropy(self, y_true, y_pred):
    return K.sum(K.binary_crossentropy(y_pred, y_true), axis=-1)


def vae_loss(self, x, x_decoded_mean):
    z_mean = self.z_m
    z_sigma = self.z_s

    # 1項目の計算
    latent_loss =  - 0.5 * K.mean(K.sum(1 + K.log(K.square(z_sigma)) - K.square(z_mean) - K.square(z_sigma), axis=-1))
    # 2項目の計算
    reconst_loss = K.mean(self.binary_crossentropy(x, x_decoded_mean),axis=-1)

    return latent_loss　+ reconst_loss

def model_compile(self, model):
    model.compile(optimizer='rmsprop', loss=self.vae_loss)

SyntaxError: invalid character in identifier (<ipython-input-4-9e9dafb6cf2d>, line 15)